In [1]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Models to do transfer learning on
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

from model_class import Model



2023-12-14 14:08:23.180110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = Model()

In [3]:
PATH_TO_FLOWERS = "../data/flowers"

In [4]:
data = model.load_data(PATH_TO_FLOWERS)

In [5]:
data = model.convert_images_column_to_tensor(data)

In [6]:
model.encode_labels(data)

,images,labels
0,"(((tf.Tensor([ 81. 103. 28.], shape=(3,), dty...",0
1,"(((tf.Tensor([30. 35. 3.], shape=(3,), dtype=...",0
2,"(((tf.Tensor([0. 0. 0.], shape=(3,), dtype=flo...",0
3,"(((tf.Tensor([22. 37. 12.], shape=(3,), dtype=...",0
4,"(((tf.Tensor([48. 58. 49.], shape=(3,), dtype=...",0
...,...,...
4312,"(((tf.Tensor([249. 252. 106.], shape=(3,), dty...",3
4313,"(((tf.Tensor([79. 88. 92.], shape=(3,), dtype=...",3
4314,"(((tf.Tensor([46. 62. 16.], shape=(3,), dtype=...",3
4315,"(((tf.Tensor([222. 228. 243.], shape=(3,), dty...",3


In [7]:
X, y = model.split_target_from_features(data)

In [8]:
X_train, X_test, X_val, y_train, y_test, y_val = model.split_train_test_val(X, y)

In [9]:
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

((2762,), (864,), (691,), (2762,), (864,), (691,))

In [10]:
X_train, X_test, X_val = model.reshape_images(X_train, X_test, X_val)

In [11]:
X_train.shape, X_test.shape, X_val.shape

((2762, 224, 224, 3), (864, 224, 224, 3), (691, 224, 224, 3))

In [12]:
train_images = tf.keras.applications.vgg19.preprocess_input(X_train)
test_images = tf.keras.applications.vgg19.preprocess_input(X_test)
val_images = tf.keras.applications.vgg19.preprocess_input(X_val)

In [13]:
y_train.shape, y_test.shape, y_val.shape

((2762,), (864,), (691,))

In [14]:
y_train = to_categorical(y_train)   
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [15]:
def create_vgg19():
    vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    vgg19.trainable = False
    model = Sequential()
    model.add(vgg19)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

In [16]:
tf.version.VERSION

'2.15.0'

In [17]:
vgg19 = create_vgg19()

In [18]:
vgg19.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 26448453 (100.89 MB)
Trainable params: 6424069 (24.51 MB)
Non-trainable params: 20024384 (76.39 MB)
_________________________________________________________________


In [19]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',      
    patience=5,             
    min_delta=0.001,        
    mode='min',             
    restore_best_weights=True 
)

In [20]:
history = vgg19.fit(train_images, y_train, epochs=20, validation_data=(val_images, y_val), callbacks=[early_stopping])

Epoch 1/20
87/87 [==============================] - 352s 4s/step - loss: 4.7809 - accuracy: 0.7487 - val_loss: 0.8640 - val_accuracy: 0.8162
Epoch 2/20
87/87 [==============================] - 343s 4s/step - loss: 0.2280 - accuracy: 0.9337 - val_loss: 0.8255 - val_accuracy: 0.8538
Epoch 3/20
87/87 [==============================] - 341s 4s/step - loss: 0.0539 - accuracy: 0.9841 - val_loss: 0.7944 - val_accuracy: 0.8625
Epoch 4/20
87/87 [==============================] - 319s 4s/step - loss: 0.0171 - accuracy: 0.9953 - val_loss: 0.8231 - val_accuracy: 0.8625
Epoch 5/20
87/87 [==============================] - 319s 4s/step - loss: 0.0116 - accuracy: 0.9978 - val_loss: 0.8553 - val_accuracy: 0.8611
Epoch 6/20
87/87 [==============================] - 322s 4s/step - loss: 0.0066 - accuracy: 0.9982 - val_loss: 0.8664 - val_accuracy: 0.8596
Epoch 7/20
87/87 [==============================] - 320s 4s/step - loss: 0.0036 - accuracy: 0.9989 - val_loss: 0.8714 - val_accuracy: 0.8596
Epoch 8/20
87

In [21]:
y_train.shape

(2762, 5)

In [22]:
vgg19.evaluate(test_images, y_test)

27/27 [==============================] - 78s 3s/step - loss: 0.7163 - accuracy: 0.8542


[0.7162773013114929, 0.8541666865348816]

## Finetune

In [23]:
def create_fine_tuned_vgg19():
    vgg19 = VGG19(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

    fine_tune_at = 2

    for layer in vgg19.layers[:fine_tune_at]:
        layer.trainable = False

    model = Sequential()
    model.add(vgg19)
    model.add(Conv2D(128, (3, 3), activation="relu"))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(
        Dense(
            128, activation="relu"
        )
    )
    model.add(
        Dense(256, activation="relu")
    )
    model.add(Dense(5, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics=["accuracy"],
    )
    return model

In [24]:
tuned_vgg19 = create_fine_tuned_vgg19()

In [25]:
tuned_vgg19.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 conv2d (Conv2D)             (None, 5, 5, 128)         589952    
                                                                 
 max_pooling2d (MaxPooling2  (None, 2, 2, 128)         0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 2, 2, 128)         0         
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               65664     
                                                      

In [26]:
tuned_history = tuned_vgg19.fit(train_images, y_train, epochs=20, validation_data=(val_images, y_val), callbacks=[early_stopping])

Epoch 1/20
87/87 [==============================] - 994s 11s/step - loss: 2.2385 - accuracy: 0.3204 - val_loss: 1.2803 - val_accuracy: 0.4269
Epoch 2/20
87/87 [==============================] - 992s 11s/step - loss: 1.2670 - accuracy: 0.4479 - val_loss: 1.3828 - val_accuracy: 0.3806
Epoch 3/20
87/87 [==============================] - 977s 11s/step - loss: 1.1744 - accuracy: 0.5217 - val_loss: 1.0900 - val_accuracy: 0.5384
Epoch 4/20
87/87 [==============================] - 977s 11s/step - loss: 1.1243 - accuracy: 0.5264 - val_loss: 1.1174 - val_accuracy: 0.5369
Epoch 5/20
87/87 [==============================] - 970s 11s/step - loss: 1.0752 - accuracy: 0.5587 - val_loss: 1.0342 - val_accuracy: 0.5861
Epoch 6/20
87/87 [==============================] - 991s 11s/step - loss: 1.0322 - accuracy: 0.5804 - val_loss: 1.0358 - val_accuracy: 0.5904
Epoch 7/20
87/87 [==============================] - 1100s 13s/step - loss: 1.0224 - accuracy: 0.5815 - val_loss: 1.0989 - val_accuracy: 0.5499
Epoch

In [27]:
tuned_vgg19.evaluate(test_images, y_test)

27/27 [==============================] - 84s 3s/step - loss: 1.0130 - accuracy: 0.6088


[1.0130324363708496, 0.6087962985038757]

In [28]:
tuned_vgg19.predict(test_images)

27/27 [==============================] - 109s 4s/step


array([[3.6889080e-02, 5.9533417e-01, 6.5580695e-03, 3.2265353e-01,
        3.8564950e-02],
       [3.2345644e-01, 1.8740186e-02, 5.6533940e-02, 1.9832566e-01,
        4.0294373e-01],
       [5.0349259e-03, 9.1640581e-04, 2.6032805e-01, 9.8380903e-03,
        7.2388250e-01],
       ...,
       [8.3159044e-02, 3.4661740e-02, 2.7340946e-01, 3.0884725e-01,
        2.9992247e-01],
       [1.6272813e-03, 7.0567730e-05, 1.3759729e-01, 2.4814412e-04,
        8.6045671e-01],
       [5.4612823e-02, 1.3334133e-02, 5.1244223e-01, 8.5116506e-02,
        3.3449435e-01]], dtype=float32)

In [29]:
y_test

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]], dtype=float32)